In [1]:
import numpy as np 
import tensorflow as tf
tf.enable_eager_execution()

config = tf.compat.v1.ConfigProto() #GPU 
config.gpu_options.allow_growth=True #GPU
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



In [2]:
x = np.arange(15).reshape(3,5)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [3]:
random = np.random.uniform(size=(3,5))
epsilon = 0.5
random_actions = np.random.randint(4, size=(3,5))
random

array([[0.51931089, 0.17677025, 0.91360996, 0.3130684 , 0.11607793],
       [0.65878392, 0.93012994, 0.15188893, 0.4604221 , 0.46544682],
       [0.03531499, 0.21437109, 0.10861807, 0.77242365, 0.12825606]])

In [4]:
np.where(random < epsilon, x, -random_actions-1)

array([[-3,  1, -4,  3,  4],
       [-3, -2,  7,  8,  9],
       [10, 11, 12, -3, 14]])

In [5]:

def mask(x, idxs):
    mask = np.zeros_like(x)
    for i in range(mask.shape[0]):
        mask[i] = idxs[i]
    return mask * x


In [6]:
class cnn_keras(tf.keras.Model):
    def __init__(self):
        tf.keras.Model.__init__(self)
        self.conv1 = tf.keras.layers.Conv2D(32,[3,3], [1,1], activation=tf.keras.activations.relu)
        self.conv2 = tf.keras.layers.Conv2D(32,[3,3], [1,1], activation=tf.keras.activations.relu)
        self.conv3 = tf.keras.layers.Conv2D(32,[3,3], [1,1], activation=tf.keras.activations.relu)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(512, tf.keras.activations.relu)
        
        
    def call(self, x):
        x = x / 255
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

In [7]:
class cnn_softmax(tf.keras.Model):
    def __init__(self, cnn):
        tf.keras.Model.__init__(self)
        self.cnn = cnn()
    
    def call(self, x):
        return tf.nn.softmax(self.cnn(x))


In [8]:
mnist = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print('x_train', x_train.shape)

x_train (50000, 32, 32, 3)


In [12]:


model = cnn_softmax(cnn_keras)

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=5)

# model.evaluate(x_test, y_test)
optimiser = tf.keras.optimizers.Adam()
cross_entr = tf.keras.losses.CategoricalCrossentropy()
batch_size = 1000
for epoch in range(5):
    avg_loss = 0.0
    for i in range(0,x_train.shape[0],1000):
        with tf.GradientTape() as tape:
            loss = cross_entr(y_train[i:i+1000], model(x_train[i:i+1000]))
            
        vars = model.trainable_variables
        grads = tape.gradient(loss, vars)
        grads, _ = tf.clip_by_global_norm(grads, 0.5)
        optimiser.apply_gradients(zip(grads,vars))
        avg_loss += tf.keras.backend.eval(loss)
        
        if i % 10000 == 0 and i >0 :
            print('epoch {}, step {} avg_loss {} '.format(epoch,i,avg_loss/i), end="\r")